In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df=pd.read_csv("F:/ml-nids-far-reduction/dataset/02-21-2018_ddos_hoic.csv",low_memory=False)

df.columns=df.columns.str.strip()
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,6,21/02/2018 08:33:25,37953,5,3,135,127,135,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,500,17,21/02/2018 08:33:06,117573474,3,0,1500,0,500,500,...,8,0.0,0.0,0,0,58800000.0,23800000.0,75600000,42000000,Benign
2,500,17,21/02/2018 08:33:06,117573474,3,0,1500,0,500,500,...,8,0.0,0.0,0,0,58800000.0,23800000.0,75600000,42000000,Benign
3,500,17,21/02/2018 08:33:11,99743998,5,0,2500,0,500,500,...,8,4000290.0,0.0,4000290,4000290,31900000.0,37900000.0,75600000,7200397,Benign
4,500,17,21/02/2018 08:33:11,99743999,5,0,2500,0,500,500,...,8,4000286.0,0.0,4000286,4000286,31900000.0,37900000.0,75600000,7200399,Benign


In [5]:
df['Label']=df['Label'].astype(str).str.strip()
df['Label']=df['Label'].apply(lambda x:0 if x=='Benign' else 1)

In [6]:
df.replace([np.inf,-np.inf],np.nan,inplace=True)
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.isna().sum().sum()

np.int64(0)

In [7]:
df.drop(columns=['Timestamp'],inplace=True) #removing non-numeric columns

In [8]:
df['Label'].value_counts(normalize=True)

Label
1    0.650028
0    0.349972
Name: proportion, dtype: float64

In [9]:
benign_df=df[df['Label']==0] #creating separate subsets for each label
attack_df=df[df["Label"]==1]
print(len(benign_df),len(attack_df))

360827 670191


In [10]:
attack_sample=attack_df.sample(n=int(len(benign_df)*0.18),random_state=42) #intentionally reversing imbalance
df=pd.concat([benign_df,attack_sample])

In [11]:
df=df.sample(frac=1,random_state=42)
df["Label"].value_counts(normalize=True)

Label
0    0.847459
1    0.152541
Name: proportion, dtype: float64

In [3]:
df['Label'].value_counts().plot(kind='bar')
plt.title("Benign vs attack traffic")
plt.xlabel("Class (Benign=0,attack=1)")
plt.ylabel("count")
plt.savefig("figures/class_distribution.png")

NameError: name 'df' is not defined

In [ ]:
variance=df.var(numeric_only=True).sort_values(ascending=False)[:15]
variance.plot(kind="barh")
plt.title("top-15-high-variance-features")
plt.savefig("figures/top_variance_features.png")


In [ ]:
df=df.drop(columns=['Dst Port'],errors=False) #Dst port is not behavoiural feature but service feature

In [ ]:
df.to_pickle("dataset/cic_processed.pkl")